In [ ]:
!python --version
print("---")

# install and import modules
%pip install rdflib

import google
import pandas as pd
import rdflib
import os
import typing

# mount drive here to read files from the folder "My Drive > Colab_Notebooks > Formal_Ontology_of_Mathematics > creativity"
google.colab.drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Colab_Notebooks/Formal_Ontology_of_Mathematics/creativity")

print("---")
!pwd

print("---")
!ls

# ACTIVATION POTENTIAL

In [ ]:
# parameters
file_name = "ontology_output_v2.ttl"


direct_history_preface = {
    "direct_history_definitions.sparql",
    "direct_history_postulates.sparql",
    "direct_history_common_notions.sparql"
}

hierarhical_history_preface = {
    "hierarchical_history_definitions.sparql",
    "hierarchical_history_postulates.sparql",
    "hierarchical_history_common_notions.sparql"
}

indirect_mereological_history_medium_importance_preface = {
    "horizontal_history_part1_definitions.sparql",
    "horizontal_history_part1_postulates.sparql",
    "horizontal_history_part1_common_notions.sparql"
}

hebbian_connections_preface = {
    "hebbian_connections_definitions.sparql",
    "hebbian_connections_postulates.sparql",
    "hebbian_connections_common_notions.sparql"
}

In [ ]:
# general functions

def access_graph(file_name: str,
                 folder_name: str = "input") -> rdflib.Graph:
    """Accesses the RDF graph from the specified file.

    Args:
        file_name: The name of the file containing the RDF graph (e.g., "ontology_output.ttl").

    Returns:
        An rdflib.Graph object representing the RDF graph.
    """
    input_file = os.path.join(folder_name, file_name)
    return rdflib.Graph().parse(input_file)


def run_sparql_query(knowledge_graph: rdflib.Graph,
                     sparql_query_name: str,
                     folder_name: str = os.path.join("input", "sparql_queries")
                     ) -> rdflib.query.Result:
    """Runs a SPARQL query on the provided knowledge graph.

    Args:
        knowledge_graph: The rdflib.Graph object representing the knowledge graph.
        sparql_query_name: The name of the SPARQL query file (e.g., "query_6.sparql").
        folder_name: The folder containing the SPARQL query file. Defaults to "input/sparql_queries".

    Returns:
        The result of the SPARQL query as an rdflib.query.Result object.
    """
    # create path to sparql query
    query_path = os.path.join(folder_name, sparql_query_name)

    # access the sparql query and run it on the knowledge graph
    with open(query_path, "r") as query_file:
        sparql_query = query_file.read()
    return knowledge_graph.query(sparql_query)


def get_table_history_queries(knowledge_graph: rdflib.Graph,
                              sparql_queries: set = direct_history_preface
                              ) -> pd.DataFrame:
    all_results = []
    # Run SPARQL queries and append results to the list
    for sparql_query in sparql_queries:
        sparql_results = run_sparql_query(knowledge_graph, sparql_query)
        for result in sparql_results:
            all_results.append([
                getattr(result.o, "toPython", lambda: result.o)(),  # Use getattr with default lambda
                int(result.links)
            ])
    # Return the pandas DataFrame from the list of results
    results = pd.DataFrame(all_results,
                        columns=["conceptual_item", "use_number"])
    return results

def get_table_hebbian_queries(knowledge_graph: rdflib.Graph,
                              sparql_queries: set = direct_history_preface
                              ) -> pd.DataFrame:
    all_results = []
    # Run SPARQL queries and append results to the list
    for sparql_query in sparql_queries:
        sparql_results = run_sparql_query(knowledge_graph, sparql_query)
        for result in sparql_results:
            all_results.append([
                getattr(result.o1, "toPython", lambda: result.o1)(),  # Use getattr with default lambda
                getattr(result.o2, "toPython", lambda: result.o2)(),  # Use getattr with default lambda
                int(result.links)
            ])
    # Return the pandas DataFrame from the list of results
    results = pd.DataFrame(all_results,
                        columns=["conceptual_item_1", "conceptual_item_2", "use_number"])
    return results


def get_table_with_links(knowledge_graph: rdflib.Graph,
                         sparql_queries: set = direct_history_preface,
                         history_table_option: bool = True
                         ) -> pd.DataFrame:
    """Retrieves a table of textual units, conceptual items, and their usage numbers.

    Executes a set of SPARQL queries on the knowledge graph to extract links between
    textual units and conceptual items, along with their usage numbers.

    Args:
        knowledge_graph: The rdflib.Graph object representing the knowledge graph.
        sparql_queries: A list of SPARQL query names to execute. Defaults to sparql_queries_withouth_hierarchical_imports.

    Returns:
        A pandas DataFrame with columns "textual_unit", "conceptual_item", and "use_number".
    """
    if history_table_option:
        results = get_table_history_queries(knowledge_graph, sparql_queries)
    else:
        results = get_table_hebbian_queries(knowledge_graph, sparql_queries)

    # Order results
    results = results.sort_values(by=["use_number"], ascending=[False])

    return results

## HISTORICAL ACTIVATION POTENTIAL

In [ ]:
# def get_direct_history(kg: rdflib.Graph,
#                        sparql_queries_folder: str = os.path.join("input", "sparql_queries")):
#     pass
    # get_direct_history_definitions()
    # get_direct_history_postulates()
    # get_direct_history_common_notions()

    # return direct_history

def get_preface_history(kg: rdflib.Graph,
                        direct_history_preface: set = direct_history_preface,
                        hierarhical_history_preface: set = hierarhical_history_preface,
                        indirect_mereological_history_medium_importance_preface: set = indirect_mereological_history_medium_importance_preface,
                        sparql_queries_folder: str = os.path.join("input", "sparql_queries")):
    # preface history: definition, postulates, common axioms
    direct_history_preface_df = get_table_with_links(kg, direct_history_preface)

    # get indirect hierarchical history
    hierachical_history_preface_df = get_table_with_links(kg, hierarhical_history_preface)

    # get indirect mereological history
    indirect_mereological_history_preface_df = get_table_with_links(kg, indirect_mereological_history_medium_importance_preface)

    return [direct_history_preface_df, hierachical_history_preface_df, indirect_mereological_history_preface_df]

def historical_activation_computation(historical_activation_potential: dict,
                                      specific_history: pd.DataFrame,
                                      weight: float):
    total = specific_history["use_number"].sum()
    for index in specific_history.index:
        conceptual_item = specific_history["conceptual_item"][index]
        historical_activation_potential[conceptual_item] += ( (weight * specific_history["use_number"][index]) / total )
    return historical_activation_potential

def get_historical_activation_potential(history: list,
                                        weight_direct: float = 6/9,
                                        weight_hierarchical: float = 1/9,
                                        weight_mereological: float = 2/9):
    # initialize the dictionary to compute the historical activation potential
    historical_activation_potential = {
        conceptual_item: 0
            for specific_history in history
            for conceptual_item in specific_history["conceptual_item"]
    }
    # add the use numbers from the three histories
    for specific_history in history:
        activation = historical_activation_computation(
            historical_activation_potential, specific_history, weight_direct)

    # concert the dictionary to a dataframe
    historical_activation_potential = pd.DataFrame(
            list(historical_activation_potential.items()),
            columns=['conceptual_item', 'activation_potential']
        )

    return historical_activation_potential



def get_history(kg: rdflib.Graph,
                sparql_queries_folder: str = os.path.join("input", "sparql_queries"),
                up_to_proposition: int = 0,
                base: dict = {}):
    if base:
        pass
        # find highest proposition P in base
        # if P < up_to_proposition,
        # find the history of the propositions
        # between P (excluded) and up_to_proposition (included)

        # if P = up_to_proposition,
        # find the history of the propositions
        # between P (excluded) and up_to_proposition (included)

        # if P > up_to_proposition,
        # remove the history for propositions > up_to_proposition
        # return history
    else:
        # get direct history of definitions, postulates, and common notions

        # get direct history up to the given proposition number
        if up_to_proposition == 0:
            history = get_preface_history(kg)
            return history
        elif up_to_proposition > 1:
            pass
            # return history
        else:
            return ValueError

## HEBBIAN ACTIVATION POTENTIAL

In [ ]:
# def get_hebbian_table(kg: rdflib.Graph,
#                       sparql_queries_folder: str = os.path.join("input", "sparql_queries"),
#                       up_to_proposition: int = 0):
#     # get_hebbian_connections
#     hebbian_connections = get_table_with_links(kg, hebbian_connections_preface, history_table_option = False)

#     return hebbian_connections


def compute_hebbian_activation(hebbian_table: pd.DataFrame):
    # initialize dictionary for hebbian activation potentials
    hebbian_potential = {}
    # sum of all hebbian connections
    total_use_number = hebbian_table['use_number'].sum()
    # add hebbian activation potentials to the dictionary
    for row in hebbian_table.itertuples():
        concepts = tuple(sorted([row.conceptual_item_1, row.conceptual_item_2]))
        hebbian_activation = row.use_number / total_use_number
        hebbian_potential[concepts] = hebbian_activation

    hebbian_df = pd.DataFrame(list(hebbian_potential.items()),
                              columns=["conceptual_item", "activation_potential"])
    return hebbian_df

def get_hebbian_activation(kg: rdflib.Graph,
                           sparql_queries_folder: str = os.path.join("input", "sparql_queries"),
                           sparql_queries: set = hebbian_connections_preface,
                           up_to_proposition: int = 0,
                           base: dict = {}):
    if base:
        pass
    else:
        if up_to_proposition == 0:
            hebbian_table = get_table_with_links(kg, sparql_queries, history_table_option = False)
            hebbian_potential = compute_hebbian_activation(hebbian_table)
            return hebbian_potential
        elif up_to_proposition > 1:
            pass
        else:
            return ValueError

# UNIFIED ACTIVATION POTENTIAL

In [ ]:
def get_context(historical_activation_potential: pd.DataFrame,
                hebbian_activation_potential: pd.DataFrame):
    context = set(historical_activation_potential['conceptual_item'])
    for pair in hebbian_activation_potential['conceptual_item']:
        context.update(pair)
    return context

def calculate_activation(conceptual_item,
                         historical_activation_potential_dict: dict,
                         hebbian_activation_potential_dict: dict,
                         context_length: int,
                         history_weight: float = 2/3,
                         hebbian_weight: float = 1/3):
    # history part
    weigthed_historical_activation = history_weight * historical_activation_potential_dict.get(conceptual_item, 0)
    # hebbian part
    weighted_hebbian_activation = 0
    for item, value in hebbian_activation_potential_dict.items():
        if conceptual_item in item:
            weighted_hebbian_activation += hebbian_weight * (hebbian_activation_potential_dict[item] / context_length)
    # total activation potential
    return weigthed_historical_activation + weighted_hebbian_activation

def get_activation_potential(historical_activation_potential: pd.DataFrame,
                             hebbian_activation_potential: pd.DataFrame):
    # get context
    context = get_context(historical_activation_potential, hebbian_activation_potential)
    # create dictionaries
    historical_activation_potential_dict = dict(zip(historical_activation_potential['conceptual_item'],
                                                    historical_activation_potential['activation_potential']))
    hebbian_activation_potential_dict = dict(zip(hebbian_activation_potential['conceptual_item'],
                                                 hebbian_activation_potential['activation_potential']))

    # initialize dictionary of unified activation potentials
    activations = {}

    # calculate the unified activation potential for each conceptual item
    context_length = len(hebbian_activation_potential)
    for conceptual_item in context:
        activations[conceptual_item] = calculate_activation(
                conceptual_item,
                historical_activation_potential_dict,
                hebbian_activation_potential_dict,
                context_length
            )
    activations_df = pd.DataFrame(list(activations.items()),
                                  columns=["conceptual_item", "activation_potential"])
    return activations_df.sort_values(by=["activation_potential"], ascending = False).reset_index(drop = True)

## MAIN

In [ ]:
def main(file_name: str):
    # access turtle file
    kg = access_graph(file_name)

    ###########
    # preface
        # history
    history = get_history(kg)
    historical_activation_potential = get_historical_activation_potential(history)
        # hebbian
    hebbian_activation_potential = get_hebbian_activation(kg)
    print(hebbian_activation_potential)
        # activation potential
    activation_potential = get_activation_potential(
            historical_activation_potential,
            hebbian_activation_potential
        )

    return activation_potential


activation_potential = main(file_name)

In [ ]:
activation_potential[:5]

In [ ]:
activation_potential.to_csv("output/activation_potential.csv", index = False)